In [1]:
from mosestokenizer import MosesTokenizer
import tokenize_uk # tokenize_words, tokenize_sents, tokenize_text
import difflib
from IPython.display import display, HTML
from compare_tokenizers_utils import display_spaces, display_diff

In [38]:
%%bash
# download data, cell not tested:
mkdir -p data
cd data
# wget https://ufallab.ms.mff.cuni.cz/~popel/uk-cs/data/train-set/full8M.uk.gz
# gzip -d full8M.uk.gz

1


In [21]:
# PURE EVIL = add uk prefixes to the installed moses package
import os
path = os.path.abspath(mosestokenizer.__file__)
path = os.path.dirname(path)
path = os.path.join(path, "nonbreaking_prefixes", "nonbreaking_prefix.uk")
print(path)
os.system(f"curl https://raw.githubusercontent.com/ufal/lindat-translation/master/app/text_utils/non_breaking_prefixes/uk.txt > {path}")
if os.path.exists(path):
    print("uk prefixes successfully installed")

/home/balhar/.local/lib/python3.10/site-packages/mosestokenizer/nonbreaking_prefixes/nonbreaking_prefix.uk
uk prefixes successfully installed


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1792  100  1792    0     0  18326      0 --:--:-- --:--:-- --:--:-- 18474


In [158]:
uk_data_path = "./data/full8M.uk.train"
lines_number = 10000
# read first lines of uk data
with open(uk_data_path) as input_file:
    head = [next(input_file).strip() for _ in range(lines_number)]

In [160]:
result = []
moses_uk = MosesTokenizer('uk')
moses_en = MosesTokenizer('en')
for line in head:
    tok_tokenize_uk = tokenize_uk.tokenize_words(line)
    tok_moses_uk = moses_uk(line)
    tok_moses_en = moses_en(line)
    result.append({
        "original": line,
        "tokenize_uk": " ".join(tok_tokenize_uk),
        "moses_uk": " ".join(tok_moses_uk),
        "moses_en": " ".join(tok_moses_en),
    })
moses_uk.close()
moses_en.close()

In [189]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [162]:
df = pd.DataFrame(result)

# How much Moses+english and Moses+ukrainian differ?

In [165]:
# total number of tested lines
len(df)

10000

In [167]:
# how often Moses with en breaking prefixes differs from Moses with uk breaking prefixes
sum(df.moses_en != df.moses_uk)

674

<h2>Legend for the detailed view</h2>
For tokenizer_A VS tokenizer_B:<br>
<span style='background-color: #FFFF88;'>yellow</span> = separating space between tokens present in both A and B<br>
<span style='background-color: #88FF88;'>green</span> = space present in B, missing in A<br>
<span style='background-color: #FF0000;'>red line</span> = space present in A, missing in B<br>
<span style='background-color: #FF88FF;'>purple</span> = replaced substring<br>

In [186]:
# show some examples for these differences
df_filtered = df[df.moses_en != df.moses_uk][:20]
diffs = df_filtered.apply(lambda row: display_diff(difflib.SequenceMatcher(None, row["moses_uk"], row["moses_en"])), axis=1)
pd.DataFrame({
    "original": df_filtered.original,
    "moses_uk": df_filtered.moses_uk,
    "moses_uk versus moses_en": diffs,
    "moses_en": df_filtered.moses_en,
}).style

,original,moses_uk,moses_uk versus moses_en,moses_en
5,За 75 секунд польоту він подолав відстань 3 км.,За 75 секунд польоту він подолав відстань 3 км.,За 75 секунд польоту він подолав відстань 3 км .,За 75 секунд польоту він подолав відстань 3 км .
18,", Відвідайте наш Інтернет-магазин для перегляду передач, частини і аксесуари для продажу, якщо ви не можете знайти те, що ви шукаєте або потрібна допомога, зв'яжіться з нами або зателефонувати нам з понеділка по п'ятницю 8 ранку до 5 вечора EST в 877-776-4600 або 407-872-1901.",", Відвідайте наш Інтернет @-@ магазин для перегляду передач , частини і аксесуари для продажу , якщо ви не можете знайти те , що ви шукаєте або потрібна допомога , зв ' яжіться з нами або зателефонувати нам з понеділка по п ' ятницю 8 ранку до 5 вечора EST в 877 @-@ 776 @-@ 4600 або 407 @-@ 872 @-@ 1901 .",", Відвідайте наш Інтернет @-@ магазин для перегляду передач , частини і аксесуари для продажу , якщо ви не можете знайти те , що ви шукаєте або потрібна допомога , зв 'яжіться з нами або зателефонувати нам з понеділка по п 'ятницю 8 ранку до 5 вечора EST в 877 @-@ 776 @-@ 4600 або 407 @-@ 872 @-@ 1901 .",", Відвідайте наш Інтернет @-@ магазин для перегляду передач , частини і аксесуари для продажу , якщо ви не можете знайти те , що ви шукаєте або потрібна допомога , зв 'яжіться з нами або зателефонувати нам з понеділка по п 'ятницю 8 ранку до 5 вечора EST в 877 @-@ 776 @-@ 4600 або 407 @-@ 872 @-@ 1901 ."
32,"Щоб підключити варильну поверхню до мережі 380 Вольт, знадобиться мідний кабель, п'ять жив - ПВС 5х4мм - тільки з його допомогою можна зробити безпечне і надійне підключення.","Щоб підключити варильну поверхню до мережі 380 Вольт , знадобиться мідний кабель , п ' ять жив - ПВС 5х4мм - тільки з його допомогою можна зробити безпечне і надійне підключення .","Щоб підключити варильну поверхню до мережі 380 Вольт , знадобиться мідний кабель , п 'ять жив - ПВС 5х4мм - тільки з його допомогою можна зробити безпечне і надійне підключення .","Щоб підключити варильну поверхню до мережі 380 Вольт , знадобиться мідний кабель , п 'ять жив - ПВС 5х4мм - тільки з його допомогою можна зробити безпечне і надійне підключення ."
46,"Маленька Орися - дуже рухлива дитина, тому батькам треба берегти її від можливих травм.Ставши дорослою, вона часто легковажно ставиться до свого здоров'я, часто зловживає знеболюючими препаратами і енергетиками, чого робити ні в якому разі не можна.","Маленька Орися - дуже рухлива дитина , тому батькам треба берегти її від можливих травм.Ставши дорослою , вона часто легковажно ставиться до свого здоров ' я , часто зловживає знеболюючими препаратами і енергетиками , чого робити ні в якому разі не можна .","Маленька Орися - дуже рухлива дитина , тому батькам треба берегти її від можливих травм.Ставши дорослою , вона часто легковажно ставиться до свого здоров 'я , часто зловживає знеболюючими препаратами і енергетиками , чого робити ні в якому разі не можна .","Маленька Орися - дуже рухлива дитина , тому батькам треба берегти її від можливих травм.Ставши дорослою , вона часто легковажно ставиться до свого здоров 'я , часто зловживає знеболюючими препаратами і енергетиками , чого робити ні в якому разі не можна ."
52,Її справжнє ім'я – Адель Лорі Блу Едкінс.,Її справжнє ім ' я – Адель Лорі Блу Едкінс .,Її справжнє ім 'я – Адель Лорі Блу Едкінс .,Її справжнє ім 'я – Адель Лорі Блу Едкінс .
55,"Теорію гомеопатії розробив німецький лікар Самуель Ганеман наприкінці XVIII ст., коли сучасна медицина його розчарувала.","Теорію гомеопатії розробив німецький лікар Самуель Ганеман наприкінці XVIII ст. , коли сучасна медицина його розчарувала .","Теорію гомеопатії розробив німецький лікар Самуель Ганеман наприкінці XVIII ст . , коли сучасна медицина його розчарувала .","Теорію гомеопатії розробив німецький лікар Самуель Ганеман наприкінці XVIII ст . , коли сучасна медицина його розчарувала ."
56,"Для утеплення підлоги, спочатку необхідно змонтувати дерев'яні лаги, між якими потрібно прокласти утеплюва

# Observations
- english moses seems to handle apostrophe differently (joins it to next word)
- english moses separates "км.", "ст.", "р." and "є.", probably the effect of using the breaking prefixes [examples: 5, 55, 127, and 165]

# How much Tokenize_UK and Moses+ukrainian differ?

In [191]:
# how often tokenize_uk differs from Moses with english breaking prefixes
sum(df.tokenize_uk != df.moses_en)

2104

In [187]:
# how often tokenize_uk differs from Moses with uk breaking prefixes
sum(df.tokenize_uk != df.moses_uk)

2160

<h2>Legend for the detailed view</h2>
For tokenizer_A VS tokenizer_B:<br>
<span style='background-color: #FFFF88;'>yellow</span> = separating space between tokens present in both A and B<br>
<span style='background-color: #88FF88;'>green</span> = space present in B, missing in A<br>
<span style='background-color: #FF0000;'>red line</span> = space present in A, missing in B<br>
<span style='background-color: #FF88FF;'>purple</span> = replaced substring<br>

In [192]:
# show some examples for these differences
df_filtered = df[df.moses_uk != df.tokenize_uk][:20]
diffs = df_filtered.apply(lambda row: display_diff(difflib.SequenceMatcher(None, row["tokenize_uk"], row["moses_uk"])), axis=1)
pd.DataFrame({
    "original": df_filtered.original,
    "tokenize_uk": df_filtered.tokenize_uk,
    "tokenize_uk versus moses_uk": diffs,
    "moses_uk": df_filtered.moses_uk,
}).style

,original,tokenize_uk,tokenize_uk versus moses_uk,moses_uk
1,"Elizabeth січень 18, 20172017-01-18T00:01:44+00:002017-08-22T20:32:29+00:00 підприємництво","Elizabeth січень 18 , 20172017-01-18 T00 : 01 : 44 +00 : 002017-08-22 T20 : 32 : 29 +00 : 00 підприємництво","Elizabeth січень 18 , 20172017 @-@ 01 @-@ 18T00 : 01 : 44 + 00 : 002017 @-@ 08 @-@ 22T20 : 32 : 29 + 00 : 00 підприємництво","Elizabeth січень 18 , 20172017 @-@ 01 @-@ 18T00 : 01 : 44 + 00 : 002017 @-@ 08 @-@ 22T20 : 32 : 29 + 00 : 00 підприємництво"
5,За 75 секунд польоту він подолав відстань 3 км.,За 75 секунд польоту він подолав відстань 3 км .,За 75 секунд польоту він подолав відстань 3 км.,За 75 секунд польоту він подолав відстань 3 км.
11,Чисельні націонал-європейські діячі зрештою долучились до лав маоїстів.,Чисельні націонал - європейські діячі зрештою долучились до лав маоїстів .,Чисельні націонал @-@ європейські діячі зрештою долучились до лав маоїстів .,Чисельні націонал @-@ європейські діячі зрештою долучились до лав маоїстів .
12,"OnePlus 2 огляд: справжній ""Флагман вбивця""?","OnePlus 2 огляд : справжній "" Флагман вбивця "" ?","OnePlus 2 огляд : справжній "" Флагман вбивця "" ?","OnePlus 2 огляд : справжній "" Флагман вбивця "" ?"
18,", Відвідайте наш Інтернет-магазин для перегляду передач, частини і аксесуари для продажу, якщо ви не можете знайти те, що ви шукаєте або потрібна допомога, зв'яжіться з нами або зателефонувати нам з понеділка по п'ятницю 8 ранку до 5 вечора EST в 877-776-4600 або 407-872-1901.",", Відвідайте наш Інтернет - магазин для перегляду передач , частини і аксесуари для продажу , якщо ви не можете знайти те , що ви шукаєте або потрібна допомога , зв'яжіться з нами або зателефонувати нам з понеділка по п'ятницю 8 ранку до 5 вечора EST в 877-776-4600 або 407-872-1901 .",", Відвідайте наш Інтернет @-@ магазин для перегляду передач , частини і аксесуари для продажу , якщо ви не можете знайти те , що ви шукаєте або потрібна допомога , зв ' яжіться з нами або зателефонувати нам з понеділка по п ' ятницю 8 ранку до 5 вечора EST в 877 @-@ 776 @-@ 4600 або 407 @-@ 872 @-@ 1901 .",", Відвідайте наш Інтернет @-@ магазин для перегляду передач , частини і аксесуари для продажу , якщо ви не можете знайти те , що ви шукаєте або потрібна допомога , зв ' яжіться з нами або зателефонувати нам з понеділка по п ' ятницю 8 ранку до 5 вечора EST в 877 @-@ 776 @-@ 4600 або 407 @-@ 872 @-@ 1901 ."
20,Слоти з тегом 'Плеєр' (група Предмети) Увійти,Слоти з тегом ' Плеєр ' ( група Предмети ) Увійти,Слоти з тегом ' Плеєр ' ( група Предмети ) Увійти,Слоти з тегом ' Плеєр ' ( група Предмети ) Увійти
29,7.High перфорована задні двері прийняти двосекційний стиль з повністю вентиляції.,7 . High перфорована задні двері прийняти двосекційний стиль з повністю вентиляції .,7.High перфорована задні двері прийняти двосекційний стиль з повністю вентиляції .,7.High перфорована задні двері прийняти двосекційний стиль з повністю вентиляції .
31,"Фраза «Хайфа працює, Єрусалим молиться, а Тель-Авів гуляє» повністю відповідає життю міста.","Фраза « Хайфа працює , Єрусалим молиться , а Тель - Авів гуляє » повністю відповідає життю міста .","Фраза « Хайфа працює , Єрусалим молиться , а Тель @-@ Авів гуляє » повністю відповідає життю міста .","Фраза « Хайфа працює , Єрусалим молиться , а Тель @-@ Авів гуляє » повністю відповідає життю міста ."
32,"Щоб підключити варильну поверхню до мережі 380 Вольт, знадобиться мідний кабель, п'ять жив - ПВС 5х4мм - тільки з його допомогою можна зробити безпечне і надійне підключення.","Щоб підключити варильну поверхню до мережі 380 Вольт , знадобиться мідний кабель , п'ять жив - ПВС 5 х4мм - тільки з його допомогою можна зробити безпечне і надійне підключення .","Щоб підключити варильну поверхню до мережі 380 Вольт , знадобиться мідний кабель , п ' ять жив - ПВС 5х4мм - тільки з його допомогою можна зробити безпечне і надійне підключення .","Щоб підключити варильну поверхню до мережі 380 Вольт , знадобиться мідний кабель , п ' ять жив - ПВС 5х

# Observations

- [ex1] moses_uk (and moses_en) adds @ around hyphens inside words
- [ex5, 55, 58] tokenize_uk separates  "км.", "ст.", "р." similar to moses_en
- [ex18,32,46,...] tokenize_uk does not separate apostrophe inside word (eg. "здоров'я" is tokenized as a single word)
- [ex12,32]~~something is going on with apostrophes and quotation marks, but not sure what? (diff seems to think they are getting replaced but it seems to me that it is the same UTF-8 character, maybe bug)~~ the apostrophes and quotation marks are getting escaped, which we do not see in the HTML output view.
- [ex46,83] in case of multiple sentences in one example, tokenize_uk separates the dot, Moses does not.
- [ex29] on the other hand, tokenize_uk seems to separate the dot even in case of the leading number of a numbered list

# Statistics: Which tokenizer is the most granular?

In [213]:
df

,original,tokenize_uk,moses_uk,moses_en
0,Застосування замороженого полуничного порошку:,Застосування замороженого полуничного порошку :,Застосування замороженого полуничного порошку :,Застосування замороженого полуничного порошку :
1,"Elizabeth січень 18, 20172017-01-18T00:01:44+00:002017-08-22T20:32:29+00:00 підприємництво","Elizabeth січень 18 , 20172017-01-18 T00 : 01 : 44 +00 : 002017-08-22 T20 : 32 : 29 +00 : 00 підприємництво","Elizabeth січень 18 , 20172017 @-@ 01 @-@ 18T00 : 01 : 44 + 00 : 002017 @-@ 08 @-@ 22T20 : 32 : 29 + 00 : 00 підприємництво","Elizabeth січень 18 , 20172017 @-@ 01 @-@ 18T00 : 01 : 44 + 00 : 002017 @-@ 08 @-@ 22T20 : 32 : 29 + 00 : 00 підприємництво"
2,"Поле для гольфу (у межах 3 км), відкритий басейн, тенісні корти, більярд, сад готелю є ідеальним місцем для вдпочинку після напруженого дня.","Поле для гольфу ( у межах 3 км ) , відкритий басейн , тенісні корти , більярд , сад готелю є ідеальним місцем для вдпочинку після напруженого дня .","Поле для гольфу ( у межах 3 км ) , відкритий басейн , тенісні корти , більярд , сад готелю є ідеальним місцем для вдпочинку після напруженого дня .","Поле для гольфу ( у межах 3 км ) , відкритий басейн , тенісні корти , більярд , сад готелю є ідеальним місцем для вдпочинку після напруженого дня ."
3,Суддя: Джек Моватт (Шотландія),Суддя : Джек Моватт ( Шотландія ),Суддя : Джек Моватт ( Шотландія ),Суддя : Джек Моватт ( Шотландія )
4,KB2 SOFT Простий калькулятор для споживання і відключення вартості (паливо + електричний).,KB2 SOFT Простий калькулятор для споживання і відключення вартості ( паливо + електричний ) .,KB2 SOFT Простий калькулятор для споживання і відключення вартості ( паливо + електричний ) .,KB2 SOFT Простий калькулятор для споживання і відключення вартості ( паливо + електричний ) .
...,...,...,...,...
9995,Я знаю і бачу це.,Я знаю і бачу це .,Я знаю і бачу це .,Я знаю і бачу це .
9996,5-зіркові готелі Сонячного Берега,5-зіркові готелі Сонячного Берега,5 @-@ зіркові готелі Сонячного Берега,5 @-@ зіркові готелі Сонячного Берега
9997,Моя тобі порада... починай бухати як слід.,Моя тобі порада ... починай бухати як слід .,Моя тобі порада ... починай бухати як слід .,Моя тобі порада ... починай бухати як слід .
9998,"Тут ви можете написати все що вам подобається, або навпаки не дуже подобається в нашому сайті.","Тут ви можете написати все що вам подобається , або навпаки не дуже подобається в нашому сайті .","Тут ви можете написати все що вам подобається , або навпаки не дуже подобається в нашому сайті .","Тут ви можете написати все що вам подобається , або навпаки не дуже подобається в нашому сайті ."


In [208]:
# number of tokens for each tokenized set
df.applymap(lambda x: len(x.split())).describe()

,original,tokenize_uk,moses_uk,moses_en
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,10.375000,12.573100,12.744000,12.688900
std,7.813941,9.546768,9.717063,9.666595
min,1.000000,1.000000,1.000000,1.000000
25%,5.000000,6.000000,6.000000,6.000000
50%,8.000000,10.000000,10.000000,10.000000
75%,14.000000,17.000000,17.000000,17.000000
max,49.000000,64.000000,79.000000,79.000000


it seems that moses_uk is the most granular = splits into most tokens on average

In [155]:
# old cells below

In [212]:
# how many outputs are the same for all tokenizers
all_same = (df.tokenize_uk == df.moses_uk) & (df.moses_uk == df.moses_en)
sum(all_same)

7840

In [193]:
# this works: pd.DataFrame({"test": ["<b>Hello</b>"]}).style
# here I show and highlight all differences:
# all_same = (df.tokenize_uk == df.moses_uk) & (df.moses_uk == df.moses_en)
# df_filtered = df[~all_same]
# df_filtered["original_html"] = df_filtered["original"].map(display_spaces)
# df_filtered["tokenize_uk_html"] = df_filtered.apply(lambda row: display_diff(difflib.SequenceMatcher(None, row["original"], row["tokenize_uk"])), axis=1)
# df_filtered["tokenizeuk_VS_mosesuk"] = df_filtered.apply(lambda row: display_diff(difflib.SequenceMatcher(None, row["tokenize_uk"], row["moses_uk"])), axis=1)
# df_filtered["mosesen_VS_mosesuk"] = df_filtered.apply(lambda row: display_diff(difflib.SequenceMatcher(None, row["moses_uk"], row["moses_en"])), axis=1)
# df_filtered[:10].style.hide_columns(["original", "tokenize_uk", "moses_uk", "moses_en"])